In [1]:
import geopandas as gpd
import os

In [2]:
shp_dir = "./LSMD_CONT_UQ111_5174_대전"
shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

shp_path = os.path.join(shp_dir, shp_files[0])
gdf1 = gpd.read_file(shp_path, encoding='cp949')

gdf1['category'] = gdf1['MNUM'].apply(
    lambda x: '주거지역' if 'UQA1' in x else
              '상업지역' if 'UQA2' in x else
              '공업지역' if 'UQA3' in x else
              '녹지지역' if 'UQA4' in x else
              '기타'
)

daejeon_dict = {
    '30110': '동구',
    '30140': '중구',
    '30170': '서구',
    '30200': '유성구',
    '30230': '대덕구'
}

gdf1['adm_cd'] = gdf1['COL_ADM_SE'].map(daejeon_dict)
gdf1 = gdf1[['adm_cd', 'category', 'geometry']]

gdf1 = gpd.GeoDataFrame(gdf1, geometry='geometry')
gdf1 = gdf1.to_crs(epsg=4326)

In [52]:
gdf1.to_csv("대전시_토지용도.csv", index=False)

In [3]:
shp_dir = "./LSMD_CONT_UQ111_세종"
shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

shp_path = os.path.join(shp_dir, shp_files[0])
gdf2 = gpd.read_file(shp_path, encoding='cp949')

gdf2['category'] = gdf2['MNUM'].apply(
    lambda x: '주거지역' if 'UQA1' in x else
              '상업지역' if 'UQA2' in x else
              '공업지역' if 'UQA3' in x else
              '녹지지역' if 'UQA4' in x else
              '기타'
)



sejong_dict = {
    '36110': '세종특별자치시',
}

gdf2['adm_cd'] = gdf2['COL_ADM_SE'].map(sejong_dict)
gdf2 = gdf2[['adm_cd', 'category', 'geometry']]

gdf2 = gpd.GeoDataFrame(gdf2, geometry='geometry')
gdf2 = gdf2.to_crs(epsg=4326)

In [46]:
gdf2.to_csv("세종시_토지용도.csv", index=False)

In [4]:
import geopandas as gpd
import os

shp_dir = "./LSMD_CONT_UQ111_충북"
shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

shp_path = os.path.join(shp_dir, shp_files[0])
gdf3 = gpd.read_file(shp_path, encoding='cp949')

gdf3['category'] = gdf3['MNUM'].apply(
    lambda x: '주거지역' if 'UQA1' in x else
              '상업지역' if 'UQA2' in x else
              '공업지역' if 'UQA3' in x else
              '녹지지역' if 'UQA4' in x else
              '기타'
)

gdf3 = gdf3[gdf3['COL_ADM_SE'] == "43110"]

chungju_dict = {
    '43110': '청주시',
}

gdf3['adm_cd'] = gdf3['COL_ADM_SE'].map(chungju_dict)
gdf3 = gdf3[['adm_cd', 'category', 'geometry']]

gdf3 = gpd.GeoDataFrame(gdf3, geometry='geometry')
gdf3 = gdf3.to_crs(epsg=4326)

In [49]:
gdf3.to_csv("청주시_토지용도.csv", index=False)

## 행정경계

In [5]:
import pandas as pd
dj_b = pd.read_csv("./대전시_행정경계.csv")
sj_b = pd.read_csv("./세종시_행정경계.csv")
cj_b = pd.read_csv("./청주시_행정경계.csv")

## 주행거리

In [10]:
ve_bu = pd.read_csv("ve_bu_2022.csv")
ve_ca = pd.read_csv("ve_ca_2022.csv")
ve_tr = pd.read_csv("ve_tr_2022.csv")

## 확인

In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# tqdm의 pandas전용 메소드를 호출
tqdm.pandas()
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)
 
# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.progress_apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 주행거리 시각화

In [11]:
ve_bu = pd.read_csv("ve_bu_2022.csv")
ve_ca = pd.read_csv("ve_ca_2022.csv")
ve_tr = pd.read_csv("ve_tr_2022.csv")

In [12]:
# 맵 객체 생성 및 데이터 로드
map = KeplerGl(height=1000, width=1500)

# 도시 행정경계
map.add_data(data=ve_bu, name="대전시_주행거리")
map.add_data(data=ve_ca, name="세종시_주행거리")
map.add_data(data=ve_tr, name="청주시_주행거리")

# 도시 행정경계
map.add_data(data=dj_b, name="대전시")
map.add_data(data=sj_b, name="세종시")
map.add_data(data=cj_b, name="청주시")

# 맵 출력 및 상세설정
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\hjk\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

c:\Users\hjk\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

c:\Users\hjk\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

c:\Users\hjk\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of ra

KeplerGl(data={'대전시_주행거리': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

In [23]:
map.save_to_html(file_name="대세청_행정구역.html")

Map saved to 대세청_행정구역.html!


In [21]:
map.save_to_html(file_name="dj_sj_cj_map.html")

Map saved to dj_sj_cj_map.html!
